In [1]:
import requests
re=requests.get('https://www.qq.com/')
print('文本编码：（服务器使用的文本编码）', re.encoding)
print('响应状态码：（200表示成功）', re.status_code)
#print('字符串方式的响应体：（服务器响应的内容）', re.text)

文本编码：（服务器使用的文本编码） GB2312
响应状态码：（200表示成功） 200


In [2]:
import re
help(re.match)
m = re.match('foo', 'foo string')
print(m)
print(m.group())
print('----------------------')

m = re.match('fee', 'foo string')
print(m)

Help on function match in module re:

match(pattern, string, flags=0)
    Try to apply the pattern at the start of the string, returning
    a Match object, or None if no match was found.

<re.Match object; span=(0, 3), match='foo'>
foo
----------------------
None


##### match  方法只能从头开始匹配    
##### group()  可以获得匹配之后的结果
#### search是搜索字符串中首次出现的位置

In [3]:
help(re.search)

Help on function search in module re:

search(pattern, string, flags=0)
    Scan through string looking for a match to the pattern, returning
    a Match object, or None if no match was found.



In [4]:
m=re.search('foo','seafoo')
print(m)
m=re.match('foo','seafoo')
print(m)

<re.Match object; span=(3, 6), match='foo'>
None


In [5]:
#####匹配多个字符串 |
m = re.match('bat|bet|bit', 'bat')
print(m.group()) if m is not None else print('None')


####匹配任意单个字符 .
m = re.match('.end', 'kend')
print(m.group()) if m is not None else print('None')

m = re.match('.end', 'end')
print(m.group()) if m is not None else print('None')


####字符串集合 []
m = re.match('[cr][23][dp][o2]', 'c3p2')
print(m.group()) if m is not None else print('None')


##### []   与 |是不同的
m = re.match('c3po|r2d2', 'c3p2')
print(m.group()) if m is not None else print('None')

bat
kend
None
c3p2
None


In [6]:
# 获取URL数据（北京地铁数据）：http://map.amap.com/service/subway?_1469083453978&srhdata=1100_drw_beijing.json
# 你需要用到以下的包

import requests
import re
import numpy as np
r = requests.get('http://map.amap.com/service/subway?_1469083453978&srhdata=1100_drw_beijing.json')
#r.text

In [7]:
import json
temp=json.loads(r.text)
#########################
type(temp)
import pandas as pd
bs=pd.DataFrame(temp)

In [8]:
bs.head()

,s,i,l,o
0,北京市地铁,1100,"{'st': [{'rs': '270 684|270 684', 'udpx': '270...","1015,660"
1,北京市地铁,1100,"{'st': [{'rs': '312 684', 'udpx': '310 685;314...","1015,660"
2,北京市地铁,1100,"{'st': [{'rs': '905 555', 'udpx': '905 558,905...","1015,660"
3,北京市地铁,1100,"{'st': [{'rs': '878 1353', 'udpx': '876 1353;8...","1015,660"
4,北京市地铁,1100,"{'st': [{'rs': '1116 935|1116 935|1116 935', '...","1015,660"


In [9]:
pattern = re.compile('"st".*?"kn"')
line=pattern.findall(r.text)
line[0]

'"st":[{"rs":"270 684|270 684","udpx":"270 687;270 681","su":"1","udsu":"1;1","n":"金安桥","sid":"110100023339035","p":"270 684","r":"900000069871|110100023339","udsi":"900000069872015;900000069871002","t":"1","si":"110100023339035","sl":"116.163167,39.923620","udli":"900000069872;900000069871","poiid":"BV10813545","lg":"0","sp":"jin an qiao"},{"rs":"226 684","udpx":"226 681;226 687","su":"1","udsu":"1;1","n":"四道桥","sid":"900000069871003","p":"226 684","r":"900000069871","udsi":"900000069871003;900000069872014","t":"0","si":"900000069871003","sl":"116.134010,39.916030","udli":"900000069871;900000069872","poiid":"BV10813547","lg":"0","sp":"si dao qiao"},{"rs":"181 684","udpx":"180 681;180 687","su":"1","udsu":"1;1","n":"桥户营","sid":"900000069871004","p":"181 684","r":"900000069871","udsi":"900000069871004;900000069872013","t":"0","si":"900000069871004","sl":"116.125809,39.912383","udli":"900000069871;900000069872","poiid":"BV10813541","lg":"0","sp":"qiao hu ying"},{"rs":"155 711","udpx":"15

In [10]:
#for i in range(len(line)):
#    p=re.compile('"rs":".*')
#线路名
import re
p=re.compile('"ln":".*?"')
st=p.findall(line[0])
st[0][6:-1]
#p=re.compile('"n":".*?"')
#s=p.findall(line[0])
p=re.compile('"sl":".*?"')
s=p.findall(line[0])
for i in range(len(s)):
    print(tuple(map(float,s[i][6:-1].split(','))))

(116.163167, 39.92362)
(116.13401, 39.91603)
(116.125809, 39.912383)
(116.122225, 39.905138)
(116.123254, 39.89578)
(116.114443, 39.890465)
(116.100334, 39.889378)


In [11]:
def getStation(text):
    line_info={}
    st_info={}
    pattern = re.compile('"st".*?"kn"')
    line_list=pattern.findall(r.text)
    for i in range(len(line_list)):
        
        pattern=re.compile('"ln":".*?"')
        line_name=pattern.findall(line_list[i])[0][6:-1]
        
        pattern=re.compile('"rs".*?"sp"')
        temp_list=pattern.findall(line_list[i])
        st_name_list=[]
        for j in range(len(temp_list)):
            
            pattern=re.compile('"n":".*?"')
            st_name=pattern.findall(temp_list[j])[0][5:-1]
            st_name_list.append(st_name)
            
            pattern=re.compile('"sl":".*?"')
            pos=tuple(map(float,pattern.findall(temp_list[j])[0][6:-1].split(',')))
            st_info[st_name]=pos
        line_info[line_name]=st_name_list    
    return line_info,st_info

In [12]:
line,station=getStation(r.text)
line['S1线']

['金安桥', '四道桥', '桥户营', '上岸', '栗园庄', '小园', '石厂']

In [13]:
def get_n_list(line):
    def n_add(dict,s1,s2):
        list1=dict.get(s1)
        if not list1:
            list1=[]
        list1.append(s2)
        dict[s1]=list1
        return dict       

    n_info={}
    for line_name,station_list in line.items():
        for i in range(len(station_list)-1):
            s1=station_list[i]
            s2=station_list[i+1]
            n_info=n_add(n_info,s1,s2)
            n_info=n_add(n_info,s2,s1)
    return n_info
n_info=get_n_list(line)

In [14]:
# 画地铁图
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['font.sans-serif'] = ['SimHei'] 
# matplotlib.rcParams['font.family']='sans-serif'
plt.figure(figsize=(20,20))
stations_graph = nx.Graph()
stations_graph.add_nodes_from(list(station.keys()))
#nx.draw(stations_graph, station, with_labels=True, node_size=10)

In [16]:
stations_connection_graph = nx.Graph(n_info)
plt.figure(figsize=(30,30))
#nx.draw(stations_connection_graph,station,with_labels=True,node_size=10)

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

In [35]:
line['S1线'],station['金安桥'],n_info['金安桥']
station

{'金安桥': (116.163167, 39.92362),
 '四道桥': (116.13401, 39.91603),
 '桥户营': (116.125809, 39.912383),
 '上岸': (116.122225, 39.905138),
 '栗园庄': (116.123254, 39.89578),
 '小园': (116.114443, 39.890465),
 '石厂': (116.100334, 39.889378),
 '苹果园': (116.177388, 39.926727),
 '古城': (116.190337, 39.90745),
 '八角游乐园': (116.212684, 39.907442),
 '八宝山': (116.235948, 39.90744),
 '玉泉路': (116.252888, 39.907433),
 '五棵松': (116.273987, 39.907456),
 '万寿路': (116.294765, 39.907474),
 '公主坟': (116.309919, 39.907469),
 '军事博物馆': (116.321459, 39.907422),
 '木樨地': (116.337583, 39.907379),
 '南礼士路': (116.352583, 39.907234),
 '复兴门': (116.356866, 39.907242),
 '西单': (116.374072, 39.907383),
 '天安门西': (116.391278, 39.907472),
 '天安门东': (116.401216, 39.90778),
 '王府井': (116.411571, 39.908069),
 '东单': (116.41848, 39.908325),
 '建国门': (116.435806, 39.908501),
 '永安里': (116.450497, 39.908454),
 '国贸': (116.461841, 39.909104),
 '大望路': (116.475783, 39.908287),
 '四惠': (116.495456, 39.908749),
 '四惠东': (116.515664, 39.908495),
 '积水潭': (116.373126

In [33]:
#时间复杂度太高，基本很难跑出结果
def dfs_Search(line,n_info,start,end):
    if not n_info.get(start):
        print('起始站点“%s”不存在。请正确输入！'%start)
        return None
    if not n_info.get(end):
        print('目的站点“%s”不存在。请正确输入！'%end)
        return None
    path=[]
    this_station=start
    path.append(this_station)
    neighbors = n_info.get(this_station)
    node={'pre_station':'',
            'this_station':this_station,
            'neighbors':neighbors,
            'path':path}
    return dfs_next_Search(node, n_info, end)
def dfs_next_Search(node, n_info, end):
    neighbors = node.get('neighbors')
    pre_station = node.get('this_station')
    path = node.get('path')
    paths = []
    for i in range(len(neighbors)):
        this_station = neighbors[i]
        if (this_station in path):
            # 如果此站点已经在路径中，说明环路，此路不通
            return None
        if neighbors[i] == end:
            # 找到终点，返回路径
            path.append(end)
            paths.append(path)
            return paths
        else:
            neighbors_ = n_info.get(this_station).copy()
            neighbors_.remove(pre_station)
            path_ = path.copy()
            path_.append(this_station)
            new_node = {'pre_station':pre_station,
                        'this_station':this_station,
                        'neighbors':neighbors_,
                        'path':path_}
            paths_ =  dfs_next_Search(new_node, n_info, end)
            if paths_:
                paths.extend(paths_)
    return paths
paths = dfs_Search(line, n_info, '金安桥', '四道桥')
print('共有%d种路径。'%len(paths))
print(paths)
for item in paths:
    print("此路径总计%d站:"%(len(item)-1))
    print('-'.join(item))    

共有1种路径。
[['金安桥', '四道桥']]
此路径总计1站:
金安桥-四道桥


In [36]:
# 第二种算法：没有启发函数的简单宽度优先
def get_path_BFS(lines_info, neighbor_info, from_station, to_station):
    # 搜索策略：以站点数量为cost（因为车票价格是按站算的）
    # 这种情况下，站点间的坐标距离难以转化为可靠的启发函数，所以只用简单的BFS算法
    # 由于每深一层就是cost加1，所以每层的cost都相同，算和不算没区别，所以省略
    
    # 检查输入站点名称
    if not neighbor_info.get(from_station):
        print('起始站点“%s”不存在。请正确输入！'%from_station)
        return None
    if not neighbor_info.get(to_station):
        print('目的站点“%s”不存在。请正确输入！'%to_station)
        return None
    
    # 搜索节点是个dict，key=站名，value是包含路过的站点list
    nodes = {}
    nodes[from_station] = [from_station]
    
    while True:
        new_nodes = {}
        for (k,v) in nodes.items():
            neighbor = neighbor_info.get(k).copy()
            if (len(v) >= 2):
                # 不往上一站走
                pre_station = v[-2]
                neighbor.remove(pre_station)
            for station in neighbor:
                # 遍历邻居
                if station in nodes:
                    # 跳过已搜索过的节点
                    continue
                path = v.copy()
                path.append(station)
                new_nodes[station] = path
                if station == to_station:
                    # 找到路径，结束
                    return path
        nodes = new_nodes
        
    print('未能找到路径')
    return None

paths = get_path_BFS(line, n_info,  '金安桥', '清源路')
print("路径总计%d站。"%(len(paths)-1))
print(paths)
print("-".join(paths))


路径总计24站。
['金安桥', '杨庄', '西黄村', '廖公庄', '田村', '海淀五路居', '慈寿寺', '西钓鱼台', '公主坟', '莲花桥', '六里桥', '西局', '泥洼', '丰台站', '首经贸', '纪家庙', '草桥', '角门西', '公益西桥', '新宫', '西红门', '高米店北', '高米店南', '枣园', '清源路']
金安桥-杨庄-西黄村-廖公庄-田村-海淀五路居-慈寿寺-西钓鱼台-公主坟-莲花桥-六里桥-西局-泥洼-丰台站-首经贸-纪家庙-草桥-角门西-公益西桥-新宫-西红门-高米店北-高米店南-枣园-清源路
